# Import

## Packages

In [124]:
import numpy as np
import pandas as pd

import re

## Data

In [17]:
# columns definition

# col = 'district(鄉鎮市區)、rps01(交易標的)、rps02(土地區段位置建物區段門牌)、rps03(土地移轉總面積平方公尺)、rps04(都市土地使用分區)、rps05(非都市土地使用分區)、rps06(非都市土地使用編定)、rps07(交易年月日)、rps08(交易筆棟數)、rps09(移轉層次)、rps10(總樓層數)、rps11(建物型態)、rps12(主要用途)、rps13(主要建材)、rps14(建築完成年月)、rps15(建物移轉總面積平方公尺)、rps16(建物現況格局-房)、rps17(建物現況格局-廳)、rps18(建物現況格局-衛)、rps19(建物現況格局-隔間)、rps20(有無管理組織)、rps21(總價元)、rps22(單價元平方公尺)、rps23(車位類別)、rps24(車位移轉總面積平方公尺)、rps25(車位總價元)、rps26(備註)、rps27(編號)、rps28(主建物面積)、rps29(附屬建物面積)、rps30(陽台面積)、rps31(電梯)'

# col_name = []
# col_desc = []
# col = col.split('、')

# for i in range(len(col)):
#     content = col[i].split('(')    
#     col_name.append(content[0])
#     col_desc.append(content[1][:-1])

# data_desc = pd.DataFrame()
# data_desc['col_name'] = col_name
# data_desc['col_desc'] = col_desc

# data_desc.to_csv('data/data_description.csv', index=False)

In [17]:
df_21Q1_1 = pd.read_csv('data/2021Q1/A_lvr_land_A.csv', error_bad_lines=False)
df_21Q1_2 = pd.read_csv('data/2021Q1/A_lvr_land_B.csv', error_bad_lines=False)
df_21Q1_3 = pd.read_csv('data/2021Q1/A_lvr_land_C.csv', error_bad_lines=False)

b'Skipping line 80: expected 29 fields, saw 31\nSkipping line 82: expected 29 fields, saw 30\nSkipping line 241: expected 29 fields, saw 30\nSkipping line 567: expected 29 fields, saw 32\nSkipping line 774: expected 29 fields, saw 30\nSkipping line 901: expected 29 fields, saw 30\nSkipping line 920: expected 29 fields, saw 32\nSkipping line 1020: expected 29 fields, saw 30\nSkipping line 1021: expected 29 fields, saw 30\nSkipping line 1070: expected 29 fields, saw 30\nSkipping line 1086: expected 29 fields, saw 30\nSkipping line 1255: expected 29 fields, saw 31\nSkipping line 1256: expected 29 fields, saw 31\n'


In [18]:
print(df_21Q1_1.shape)
print(df_21Q1_2.shape)
print(df_21Q1_3.shape)

(6520, 33)
(205, 28)
(1327, 29)


# Data process

#### columns names setting

In [43]:
col = pd.read_csv('data/column_names.csv')
col_names = col['col_name'].tolist()

In [98]:
df = pd.read_csv('data/2021Q1/A_lvr_land_A.csv')
df = df.iloc[1:, :]
df.columns = col_names

In [99]:
df.head(3)

,district,transaction_sign,address,land_area,use_zone,non_metropolis_use_district,non_metropolis_use,yyymmdd,transaction_number,floor,...,berth_type,berth_area,berth_price,note,serial_number,main_building_area,ancillary_area,terrace_area,elevator,shift_number
1,文山區,房地(土地+建物),興隆路二段217巷1~30號,8.65,都市：其他:第三種住宅區。,NaN,NaN,1091029,土地1建物1車位0,五層,...,NaN,0.0,0,陽台外推;其他增建;,RPWOMLKKJIPFFAA87DA,25.66,0.70,5.79,有,NaN
2,文山區,房地(土地+建物),和平東路四段65巷77弄1~30號,86.0,都市：其他:第二種住宅區。,NaN,NaN,1091105,土地1建物1車位0,全,...,NaN,0.0,0,陽台外推;其他增建;,RPVPMLSKJIPFFAA68DA,133.66,35.95,6.11,無,NaN
3,文山區,建物,羅斯福路六段26巷1~30號,0.0,NaN,NaN,NaN,1091102,土地0建物1車位0,一層,...,NaN,0.0,0,NaN,RPSOMLOLJIPFFAA97DA,79.57,0.00,0.00,無,NaN


In [100]:
df.dtypes

district                        object
transaction_sign                object
address                         object
land_area                       object
use_zone                        object
non_metropolis_use_district     object
non_metropolis_use              object
yyymmdd                         object
transaction_number              object
floor                           object
total_floor                     object
building_state                  object
main_use                        object
building_materials              object
building_yyymmdd                object
building_area                   object
room                            object
hall                            object
health                          object
compartment                     object
manage                          object
total_price                     object
unit_price                      object
berth_type                      object
berth_area                      object
berth_price              

#### use zone

In [82]:
df.groupby('use_zone').size().rename('count').reset_index().sort_values(by='count', ascending=False).head()

,use_zone,count
139,都市：其他:第三種住宅區。,2822
147,都市：其他:第三種商業區。,376
242,都市：其他:第肆種商業區(依都市計畫說明書圖規定辦理，始得作第肆種商業區使用)(原屬第參種商...,258
155,都市：其他:第二種住宅區。,252
220,都市：其他:第四種住宅區。,198


In [101]:
df['residential_zone'] = np.where((df['use_zone'].str.find('住宅區')>=0)|(df['use_zone'].str.find('住宅用地')>=0), 1, 0)
df['business_zone'] = np.where(df['use_zone'].str.find('商業區')>=0, 1, 0)

#### transaction time

In [121]:
df['yyyy'] = df['yyymmdd'].str[:-4].astype(int)+1911
df['mm'] = df['yyymmdd'].str[-4:-2]

#### transaction number

In [150]:
df['land'] = 0
df['building'] = 0
df['berth'] = 0

for i in range(1, len(df)+1):
    df['land'][i] = re.split(r'(\d+)', df['transaction_number'][i])[1]
    df['building'][i] = re.split(r'(\d+)', df['transaction_number'][i])[3]
    df['berth'][i] = re.split(r'(\d+)', df['transaction_number'][i])[5]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### floor

convert chinese to Arabic digit

In [243]:
df['total_floor_ch'] = df['total_floor_ch'].str[:-1]
df['total_floor'] = 0

for i in range(1, len(df)+1):
    num = re.split(r'(\d+)', str(df['total_floor_ch'][i]).replace('十', '10'))
    num = list(map(lambda e: e.replace('一', '1').replace('二', '2').replace('三', '3').replace('四', '4').replace('五', '5')\
                  .replace('六', '6').replace('七', '7').replace('八', '8').replace('九', '9'), num))
    
    if (len(num) == 1) & (num[0] != 'nan'):
        df['total_floor'][i] = num[0]
    elif len(num) == 3:
        if num[0] == '':
            if num[2] == '':
                df['total_floor'][i] = 10
            else:
                df['total_floor'][i] = int(num[1])+int(num[2])
        else:
            if num[2] == '':
                df['total_floor'][i] = int(num[0])*int(num[1])
            else:
                df['total_floor'][i] = int(num[0])*int(num[1])+int(num[2])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7